In [13]:
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
X = df.drop('diseases', axis=1)
y = df['diseases']

In [4]:
le = LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

In [5]:
df = pd.concat([X, y], axis=1)
class_counts = y.value_counts()
valid_classes = class_counts[class_counts > 1].index
df = df[df[y.name].isin(valid_classes)]

X = df.drop(y.name, axis=1)
y = df[y.name]

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size = 0.2, stratify=y, random_state = 42)

In [10]:
model = xgb.XGBClassifier(
    n_estimators=50,
    max_depth=6,
    learning_rate=0.2,
    subsample=0.6,
    colsample_bytree=0.7,
    max_bin = 256,
    random_state=42,
    eval_metric='mlogloss',
    tree_method='gpu_hist',
    device='cuda'
)

In [11]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\Kush\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:45:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Kush\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:52:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Kush\AppData\Local\Programs\Python\Python312\Li

In [12]:
classif = classification_report(y_test, y_pred)
print(classif)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       0.96      0.98      0.97        81
           2       0.81      0.88      0.84        58
           3       0.80      1.00      0.89         4
           4       0.83      0.91      0.87        68
           5       0.75      1.00      0.86         6
           6       0.86      0.86      0.86         7
           7       0.71      0.71      0.71        17
           8       0.68      0.78      0.72        99
           9       0.85      0.76      0.81       182
          10       0.95      0.90      0.93       241
          11       0.79      0.77      0.78       243
          12       0.68      0.60      0.64       181
          13       0.00      0.00      0.00         2
          14       0.41      0.36      0.38        42
          15       0.97      0.96      0.96       182
          16       0.74      0.75      0.74       137
          17       0.90    

C:\Users\Kush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Kush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Kush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [14]:
acc = accuracy_score(y_test, y_pred)
print(acc)

0.8450775523427692


In [15]:
model.save_model("diseaseml.json")